In [88]:
import requests
import numpy as np
np.set_printoptions(precision=2)
import pandas as pd
import math

import json
import time
from datetime import datetime, timedelta
import sgp4
from sgp4.api import Satrec, WGS72
from sgp4.conveniences import jday
from sgp4.ext import invjday

from sgp4.api import accelerated

import matplotlib.pyplot as plt

from astropy.time import Time

from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation
from astropy import units as u
from astropy.coordinates import ITRS

from astropy.coordinates import EarthLocation, AltAz

# from joblib import Parallel, delayaed
from tqdm import tqdm

In [92]:
def get_tle(sat_num):
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"

    # 요청에 필요한 인증 정보
    payload = {"identity": 'asas4539@hanyang.ac.kr', "password": 'onsaemiro1729!!'}

    # Session 객체 생성
    session = requests.Session()

    # 로그인 요청
    response = session.post(url, data=payload)

    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None

    ## TLE 요청
    response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{sat_num}/orderby/EPOCH%20ASC/EPOCH/2022-12-31--2024-08-01/format/kvn"
    )

    response2 = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{sat_num}/orderby/EPOCH%20ASC/EPOCH/2022-12-31--2024-08-01/format/3le"
    )

    return response,response2

In [93]:
response,response2 = get_tle(58400)
tle_datas = response.text.split('\r\n')

In [91]:
print(tle_datas)

['0 TBA - TO BE ASSIGNED', '1 58400U 23179A   23326.01002942  .00008068  00000-0  36481-3 0  9990', '2 58400  97.4276 210.7863 0014260 279.3296  80.6327 15.21105052    66', '0 MALLIGYONG-1', '1 58400U 23179A   23326.16510777  .00018821  00000-0  84547-3 0  9990', '2 58400  97.4263 210.9434 0014062 277.7388 210.8950 15.21112775    83', '0 MALLIGYONG-1', '1 58400U 23179A   23326.86521155  .00009675  00000-0  43647-3 0  9992', '2 58400  97.4275 211.6299 0014281 275.8375  84.1232 15.21123879   195', '0 MALLIGYONG-1', '1 58400U 23179A   23326.95239345  .00014863  00000-0  66826-3 0  9998', '2 58400  97.4261 211.7175 0014282 275.5273 201.5425 15.21130424   204', '0 MALLIGYONG-1', '1 58400U 23179A   23327.21599119  .00012574  00000-0  56597-3 0  9997', '2 58400  97.4263 211.9766 0014186 274.4179 205.2075 15.21134246   242', '0 MALLIGYONG-1', '1 58400U 23179A   23327.45725493  .00009464  00000-0  42690-3 0  9992', '2 58400  97.4276 212.2137 0014294 273.4322  86.5279 15.21135991   289', '0 MALL

In [94]:
import re

In [95]:
creation_dates = re.findall(r"CREATION_DATE\s*=\s*([\d\-T:]+)", response.text)

In [96]:
temp = pd.DataFrame(creation_dates)
temp

,0
0,2023-11-22T03:26:15
1,2023-11-22T04:06:16
2,2023-11-23T03:43:51
3,2023-11-23T00:56:19
4,2023-11-23T05:56:15
...,...
1153,2024-07-31T03:53:23
1154,2024-07-31T06:56:18
1155,2024-07-31T15:06:16
1156,2024-07-31T23:06:17


In [97]:
len(creation_dates)

1158

In [98]:
creation_dates = list(creation_dates)

In [99]:
creation_dates

['2023-11-22T03:26:15',
 '2023-11-22T04:06:16',
 '2023-11-23T03:43:51',
 '2023-11-23T00:56:19',
 '2023-11-23T05:56:15',
 '2023-11-23T15:06:16',
 '2023-11-23T18:35:55',
 '2023-11-23T22:56:17',
 '2023-11-24T04:22:49',
 '2023-11-24T06:56:18',
 '2023-11-24T14:46:18',
 '2023-11-24T18:59:10',
 '2023-11-24T22:56:18',
 '2023-11-25T02:53:27',
 '2023-11-25T06:56:17',
 '2023-11-25T21:51:31',
 '2023-11-26T04:25:13',
 '2023-11-26T07:06:18',
 '2023-11-26T15:06:17',
 '2023-11-26T19:15:13',
 '2023-11-27T04:10:42',
 '2023-11-26T23:06:18',
 '2023-11-27T07:06:17',
 '2023-11-27T14:56:18',
 '2023-11-27T18:45:24',
 '2023-11-28T04:20:03',
 '2023-11-27T22:56:17',
 '2023-11-28T06:56:18',
 '2023-11-28T15:06:17',
 '2023-11-28T18:01:21',
 '2023-11-29T03:16:06',
 '2023-11-28T23:06:18',
 '2023-11-29T07:06:17',
 '2023-11-29T15:06:16',
 '2023-11-29T18:39:17',
 '2023-11-29T21:46:15',
 '2023-11-30T07:06:17',
 '2023-11-30T14:56:18',
 '2023-12-01T07:16:19',
 '2023-12-01T15:06:18',
 '2023-12-01T22:56:19',
 '2023-12-02T03:

In [100]:
info = pd.DataFrame([creation_dates , tle_datas[1:][::3],tle_datas[2:][::3]])

In [101]:
kt_sat = info.T

In [ ]:
kt_sat.columns = ['creation_date', 'line1', 'line2']

In [ ]:
kt_sat.to_csv('58400_new.csv')

In [ ]:
kt_sat.to_csv('42984_new.csv')

In [3]:
import requests
import numpy as np
import pandas as pd

In [102]:
dat=kt_sat

In [4]:
dat = pd.read_csv('58400_new.csv',index_col=0)

In [103]:
dat.columns = ['creation_date', 'first_line', 'second_line']

In [104]:
dat

,creation_date,first_line,second_line
0,2023-11-22T03:26:15,None,None
1,2023-11-22T04:06:16,None,None
2,2023-11-23T03:43:51,None,None
3,2023-11-23T00:56:19,None,None
4,2023-11-23T05:56:15,None,None
...,...,...,...
1153,2024-07-31T03:53:23,None,None
1154,2024-07-31T06:56:18,None,None
1155,2024-07-31T15:06:16,None,None
1156,2024-07-31T23:06:17,None,None


In [7]:
year = dat['creation_date'].map(lambda x: x.split('-')[0])
month = dat['creation_date'].map(lambda x: x.split('-')[1])
day = dat['creation_date'].map(lambda x: x.split('-')[2].split('T')[0])
hour = dat['creation_date'].map(lambda x: x.split('T')[1].split(':')[0])
minute = dat['creation_date'].map(lambda x: x.split(':')[1])
second = dat['creation_date'].map(lambda x: x.split(':')[2])

In [8]:
dat['year'] = year
dat['month'] = month
dat['day'] = day
dat['hour'] = hour
dat['minute'] = minute
dat['second'] = second

In [9]:
dat =dat[['year', 'month', 'day', 'hour', 'minute', 'second', 'first_line', 'second_line', 'creation_date']]

In [10]:
dat = dat.sort_values(by = 'creation_date').reset_index(drop=True)

In [11]:
dat

,year,month,day,hour,minute,second,first_line,second_line,creation_date
0,2023,11,22,03,26,15,1 58400U 23179A 23326.01002942 .00008068 0...,2 58400 97.4276 210.7863 0014260 279.3296 80...,2023-11-22T03:26:15
1,2023,11,22,04,06,16,1 58400U 23179A 23326.16510777 .00018821 0...,2 58400 97.4263 210.9434 0014062 277.7388 210...,2023-11-22T04:06:16
2,2023,11,23,00,56,19,1 58400U 23179A 23326.95239345 .00014863 0...,2 58400 97.4261 211.7175 0014282 275.5273 201...,2023-11-23T00:56:19
3,2023,11,23,03,43,51,1 58400U 23179A 23326.86521155 .00009675 0...,2 58400 97.4275 211.6299 0014281 275.8375 84...,2023-11-23T03:43:51
4,2023,11,23,05,56,15,1 58400U 23179A 23327.21599119 .00012574 0...,2 58400 97.4263 211.9766 0014186 274.4179 205...,2023-11-23T05:56:15
...,...,...,...,...,...,...,...,...,...
1153,2024,07,31,03,53,23,1 58400U 23179A 24212.92062143 .00008307 0...,2 58400 97.3982 99.0288 0003275 70.8065 289...,2024-07-31T03:53:23
1154,2024,07,31,06,56,18,1 58400U 23179A 24213.18365506 .00008690 0...,2 58400 97.3982 99.2875 0003254 70.4061 289...,2024-07-31T06:56:18
1155,2024,07,31,15,06,16,1 58400U 23179A 24213.57820414 .00007822 0...,2 58400 97.3981 99.6755 0003225 69.4820 290...,2024-07-31T15:06:16
1156,2024,07,31,23,06,17,1 58400U 23179A 24213.84123612 .00009173 0...,2 58400 97.3981 99.9341 0003211 69.0697 291...,2024-07-31T23:06:17
